## Importer les fichiers et les modules

In [2]:
import pandas as pd
from edc import cvrp
from edc import processing
from edc import utils
import datetime
import itertools
import numpy as np

%load_ext autoreload
%autoreload 2

## Méthode 1 : algorithme du voyageur de commerce

In [2]:
df_routes = pd.read_csv("../data/routes_v2.csv")

In [3]:
df_routes["stops_list"] = df_routes.apply(lambda row : row.stops.split(' > '),axis=1)

In [6]:
df_routes_optimisees = pd.read_csv("..\data\\intermediate_data\df_routes_optimisees.csv")

In [7]:
df_routes_optimisees.trajectoires_optimisees = df_routes_optimisees.trajectoires_optimisees.apply(lambda row : row.replace('[','').replace(']','').split(","))

## Méthode 2 : Vehicule Routing Problem

In [3]:
df_cities = pd.read_csv("../data/cities.csv")
df_orders = pd.read_csv("../data/orders.csv")
df_routes = pd.read_csv("../data/routes.csv")
df_trucks = pd.read_csv("../data/trucks.csv")
df_routes_orders = pd.read_csv("../data/intermediate_data/df_routes_orders.csv")

In [4]:
df_orders.delivered_date = pd.to_datetime( df_orders.delivered_date )

In [105]:
df_1 = pd.merge(df_orders,
                df_cities[["city","lat","lng"]],
                left_on="from_warehouse",
                right_on="city"
                ).rename(columns={'lat':'lat_warehouse','lng':'lng_warehouse'}).drop(columns=["city"])
df_orders_opt = pd.merge(df_1,
                           df_cities[["city","lat","lng"]],
                           left_on="delivery_location",
                           right_on="city").rename(columns={'lat':'lat_delivery','lng':'lng_delivery'}).drop(columns=["city"])
del(df_1)

In [5]:
date_test = datetime.datetime(2021,3,1)
delay_test = 1
warehouse_name_test = 'Cergy'

df_filtered,matrix_distance,demand,vehicle_capacities,num_vehicles,depot = processing.processing_opti_rendement(df_orders_opt,warehouse_name_test,date_test,delay_test)
lists_of_cities = cvrp.CVRP(cvrp.create_data_model(df_filtered,matrix_distance,demand,vehicle_capacities,num_vehicles,depot),verbose=0)

In [6]:
date_test = datetime.datetime(2021,4,2)
delay_test = 1
warehouse_name_test = 'Avignon'

df_filtered,matrix_distance,demand,vehicle_capacities,num_vehicles,depot = processing.processing_opti_rendement(df_orders_opt,warehouse_name_test,date_test,delay_test)
lists_of_cities = cvrp.CVRP(cvrp.create_data_model(df_filtered,matrix_distance,demand,vehicle_capacities,num_vehicles,depot),verbose=0)
lists_of_cities = [ utils.remove_consecutive_duplicate(list_of_city) for list_of_city in lists_of_cities] 

### Délai de 1 jour

In [128]:
CO2_per_km = 0.795

def calculate_nb_camions(row):
    try :
        return len([list_of_cities for list_of_cities in row if len(list_of_cities) > 1])  
    except :
        return np.nan 

def calculate_distance_trajet(row):
    try :
        return [utils.total_distance_trip(list_of_cities) + utils.dist(utils.donnees_cities(list_of_cities[-1]),utils.donnees_cities(list_of_cities[0])) for list_of_cities in row]
    except :
        np.nan  

def calculate_nb_days_1_trajet(distance,list_of_cities):
    if len(list_of_cities) == 1 :
        return 0
    else :
        temps_route = distance / 100
        temps_arrets = (len(list_of_cities) - 1) * 0.5
        return (temps_route+temps_arrets) // 8 + 1

def calculate_nb_days_trajets(row):
    try :
        return [calculate_nb_days_1_trajet(row.distance_trajet[i], row['Predicted roads'][i]) for i in range(50)]
    except :
        return np.nan

def non_zero_elements(row):
    try :
        return [element for element in row if element>0]
    except :
        return np.nan

def sum_values(row):
    try :
        return sum(row)
    except :
        return np.nan

#### Scénario 0

In [10]:
liste_warehouses_0 = ['Cergy','Reims','Avignon','Montauban','Clermont-Ferrand']
df_optimised_0 = pd.DataFrame(list(itertools.product(liste_warehouses_0,[datetime.datetime(2021,1,1) + datetime.timedelta(days=2*i) for i in range(91)])),columns=['warehouse','date'])
df_optimised_0["Predicted roads"] = df_optimised_0.apply(lambda row : cvrp.apply_CVRP(df_orders_opt,row.warehouse,row.date,1), axis=1)
df_optimised_0.to_csv("..\data\\results\df_optimised_0.csv")

No solution
No solution
No solution
No solution


In [60]:
df_optimised_0['nb_camions_totat'] = df_optimised_0['Predicted roads'].apply(calculate_nb_camions)
df_optimised_0['nb_camions_daily'] = df_optimised_0['nb_camions_totat']/181
df_optimised_0['distance_trajet'] = df_optimised_0['Predicted roads'].apply(calculate_distance_trajet)
df_optimised_0['nb_days_trajets'] = df_optimised_0.apply(calculate_nb_days_trajets,axis=1)
df_optimised_0['distance_all_trajets'] = df_optimised_0['distance_trajet'].apply(sum_values)
df_optimised_0['CO2(kg)'] = df_optimised_0['distance_all_trajets'].apply(lambda row : row * CO2_per_km)

In [118]:
df_optimised_0.groupby('warehouse').sum()

,nb_camions,distance_all_trajets,CO2(kg),nb_camions_daily
warehouse,,,,
Avignon,1071.0,589743.0,468845.685,5.884615
Cergy,2966.0,1705578.0,1355934.510,16.296703
Clermont-Ferrand,289.0,168732.0,134141.940,1.587912
Montauban,668.0,534141.0,424642.095,3.670330
Reims,831.0,735050.0,584364.750,4.565934


In [122]:
print('CO2 émis : ' +str(int(df_optimised_0.groupby('warehouse').sum()['CO2(kg)'].sum())) + ' kg')

CO2 émis : 2967928 kg


#### Scénario 1

In [11]:
df_cities_1 = pd.read_csv("../data/cities.csv")
liste_warehouses_1 = ['Cergy','Reims','Avignon','Montauban']
df_cities_1["from_warehouse"] = df_cities_1.city.apply(lambda row : processing.find_closest_warehouse(row,liste_warehouses_1))
df_orders_opt_1 = df_orders_opt.drop(columns=['from_warehouse'])
df_orders_opt_1 = pd.merge(df_orders_opt_1,df_cities_1[['city','from_warehouse']],
                           right_on='city',
                           left_on='delivery_location',
                           how='right'
                           ).drop(columns=['city'])

No solution
No solution
No solution
No solution


In [127]:
df_orders_opt_1['nb_camions_totat'] = df_orders_opt_1['Predicted roads'].apply(calculate_nb_camions)
df_orders_opt_1['nb_camions_daily'] = df_orders_opt_1['nb_camions_totat']/182
df_orders_opt_1['distance_trajet'] = df_orders_opt_1['Predicted roads'].apply(calculate_distance_trajet)
df_orders_opt_1['nb_days_trajets'] = df_orders_opt_1.apply(calculate_nb_days_trajets,axis=1)
df_orders_opt_1['distance_all_trajets'] = df_orders_opt_1['distance_trajet'].apply(sum_values)
df_orders_opt_1['CO2(kg)'] = df_orders_opt_1['distance_all_trajets'].apply(lambda row : row * CO2_per_km)

In [128]:
df_orders_opt_1.groupby('warehouse').sum()

,nb_camions_totat,nb_camions_daily,distance_all_trajets,CO2(kg)
warehouse,,,,
Avignon,1071.0,5.884615,590516.0,469460.220
Cergy,2955.0,16.236264,1647095.0,1309440.525
Montauban,669.0,3.675824,533622.0,424229.490
Reims,831.0,4.565934,736031.0,585144.645


In [129]:
print('CO2 émis : ' +str(int(df_orders_opt_1.groupby('warehouse').sum()['CO2(kg)'].sum())) + ' kg')

CO2 émis : 2788274 kg


#### Scénario 2

In [106]:
df_cities_2 = pd.read_csv("../data/cities.csv")
liste_warehouses_2 = ['Cergy','Reims','Avignon','Montauban','Rennes']
df_cities_2["from_warehouse"] = df_cities_2.city.apply(lambda row : processing.find_closest_warehouse(row,liste_warehouses_2))
df_orders_opt_2 = df_orders_opt.drop(columns=['from_warehouse'])
df_orders_opt_2 = pd.merge(df_orders_opt_2,df_cities_2[['city','from_warehouse']],right_on='city',left_on='delivery_location',how='right').drop(columns=['city'])

In [12]:
date_test = datetime.datetime(2021,3,1)
delay_test = 1
warehouse_name_test = 'Rennes'

df_filtered,matrix_distance,demand,vehicle_capacities,num_vehicles,depot = processing.processing_opti_rendement(df_orders_opt_2,warehouse_name_test,date_test,delay_test)
lists_of_cities = cvrp.CVRP(cvrp.create_data_model(df_filtered,matrix_distance,demand,vehicle_capacities,num_vehicles,depot),verbose=0)
lists_of_cities = [ utils.remove_consecutive_duplicate(list_of_city) for list_of_city in lists_of_cities] 

In [9]:
df_optimised_2 = pd.DataFrame(list(itertools.product(liste_warehouses_2,[datetime.datetime(2021,1,1) + datetime.timedelta(days=2*i) for i in range(91)])),columns=['warehouse','date'])
df_optimised_2["Predicted roads"] = df_optimised_2.apply(lambda row : cvrp.apply_CVRP(df_orders_opt_2,row.warehouse,row.date,1), axis=1)
df_optimised_2.to_csv("..\data\\results\df_optimised_2.csv")

In [10]:
df_optimised_2['nb_camions_totat'] = df_optimised_2['Predicted roads'].apply(calculate_nb_camions)
df_optimised_2['nb_camions_daily'] = df_optimised_2['nb_camions_totat']/182
df_optimised_2['distance_trajet'] = df_optimised_2['Predicted roads'].apply(calculate_distance_trajet)
df_optimised_2['nb_days_trajets'] = df_optimised_2.apply(calculate_nb_days_trajets,axis=1)
df_optimised_2['distance_all_trajets'] = df_optimised_2['distance_trajet'].apply(sum_values)
df_optimised_2['CO2(kg)'] = df_optimised_2['distance_all_trajets'].apply(lambda row : row * CO2_per_km)

In [11]:
df_optimised_2.groupby('warehouse').sum()

,nb_camions_totat,nb_camions_daily,distance_all_trajets,CO2(kg)
warehouse,,,,
Avignon,1260,6.923077,8.670176e+05,689278.977404
Cergy,2546,13.989011,8.258046e+05,656514.624564
Montauban,555,3.049451,4.138027e+05,328973.142631
Reims,864,4.747253,1.154141e+06,917542.263009
Rennes,809,4.445055,6.497294e+05,516534.838167


In [12]:
print('CO2 émis : ' +str(int(df_optimised_2.groupby('warehouse').sum()['CO2(kg)'].sum())) + ' kg')

CO2 émis : 3108843 kg


#### Scénario 3

In [27]:
df_cities_3 = pd.read_csv("../data/cities.csv")
liste_warehouses_3 = ['Cergy','Mulhouse','Avignon','Montauban','Rennes']
df_cities_3["from_warehouse"] = df_cities_3.city.apply(lambda row : processing.find_closest_warehouse(row,liste_warehouses_3))
df_orders_opt_3 = df_orders_opt.drop(columns=['from_warehouse'])
df_orders_opt_3 = pd.merge(df_orders_opt_3,df_cities_3[['city','from_warehouse']],right_on='city',left_on='delivery_location',how='right').drop(columns=['city'])

In [28]:
df_optimised_3 = pd.DataFrame(list(itertools.product(liste_warehouses_3,[datetime.datetime(2021,1,1) + datetime.timedelta(days=2*i) for i in range(91)])),columns=['warehouse','date'])
df_optimised_3["Predicted roads"] = df_optimised_3.apply(lambda row : cvrp.apply_CVRP(df_orders_opt_3,row.warehouse,row.date,1), axis=1)
df_optimised_3.to_csv("..\data\\results\df_optimised_2.csv")

In [12]:
print('CO2 émis : ' +str(int(df_optimised_3.groupby('warehouse').sum()['CO2(kg)'].sum())) + ' kg')

CO2 émis : 0 kg


### Délai de 2 jours

In [ ]:
df_optimised_0_b = pd.DataFrame(list(itertools.product(liste_warehouses_0,[datetime.datetime(2021,1,1) + datetime.timedelta(days=3*i) for i in range(60)])),columns=['warehouse','date'])
df_optimised_0_b["Predicted roads"] = df_optimised_0_b.apply(lambda row :cvrp.apply_CVRP(df_orders_opt,row.warehouse,row.date,2), axis=1)
df_optimised_0_b.to_csv("..\data\\results\df_optimised_0_b.csv")

In [ ]:
df_optimised_1_b = pd.DataFrame(list(itertools.product(liste_warehouses_1,[datetime.datetime(2021,1,1) + datetime.timedelta(days=3*i) for i in range(60)])),columns=['warehouse','date'])
df_optimised_1_b["Predicted roads"] = df_optimised_1_b.apply(lambda row :cvrp.apply_CVRP(df_orders_opt,row.warehouse,row.date,2), axis=1)
df_optimised_1_b.to_csv("..\data\\results\df_optimised_1_b.csv")

In [119]:
liste_warehouses_2 = ['Cergy','Reims','Avignon','Montauban','Rennes']
df_optimised_2_b = pd.DataFrame(list(itertools.product(liste_warehouses_2,[datetime.datetime(2021,1,1) + datetime.timedelta(days=3*i) for i in range(6)])),columns=['warehouse','date'])
df_optimised_2_b["Predicted roads"] = df_optimised_2_b.apply(lambda row :cvrp.apply_CVRP(df_orders_opt_2,row.warehouse,row.date,2), axis=1)
df_optimised_2_b.to_csv("..\data\\results\df_optimised_2_b_.csv")

No solution


In [ ]:
df_optimised_3_b = pd.DataFrame(list(itertools.product(liste_warehouses_3,[datetime.datetime(2021,1,1) + datetime.timedelta(days=i) for i in range(60)])),columns=['warehouse','date'])
df_optimised_3_b["Predicted roads"] = df_optimised_3_b.apply(lambda row :cvrp.apply_CVRP(df_orders_opt,row.warehouse,row.date,1), axis=1)
df_optimised_3_b.to_csv("..\data\\results\df_optimised_3_b.csv")

No solution
No solution
No solution
No solution


In [ ]:
df_optimised_2_b['nb_camions_totat'] = df_optimised_2_b['Predicted roads'].apply(calculate_nb_camions)
df_optimised_2_b['nb_camions_daily'] = df_optimised_2_b['nb_camions_totat']/182
df_optimised_2_b['distance_trajet'] = df_optimised_2_b['Predicted roads'].apply(calculate_distance_trajet)
df_optimised_2_b['nb_days_trajets'] = df_optimised_2_b.apply(calculate_nb_days_trajets,axis=1)
df_optimised_2_b['distance_all_trajets'] = df_optimised_2_b['distance_trajet'].apply(sum_values)
df_optimised_2_b['CO2(kg)'] = df_optimised_2_b['distance_all_trajets'].apply(lambda row : row * CO2_per_km)

In [130]:
df_optimised_2_b.groupby('warehouse').sum()

,nb_camions_totat,nb_camions_daily,distance_all_trajets,CO2(kg)
warehouse,,,,
Avignon,392.0,2.153846,254933.334508,202672.000934
Cergy,709.0,3.895604,230522.753766,183265.589244
Montauban,167.0,0.917582,122999.620183,97784.698045
Reims,244.0,1.340659,336235.982066,267307.605743
Rennes,242.0,1.329670,189989.017654,151041.269035


In [143]:
df_optimised_2_b['CO2(kg)'].sum()

2706213.489003308